In [203]:
abstract ERC20 = {
    flags startcat = Description;
    cat
        Description;
        Variable;
        BoolOperator;
        Expression;
    fun
        mkExpression: Variable -> BoolOperator -> Variable -> Expression;
        mkConditionalExpression: Expression -> BoolOperator -> Expression -> BoolOperator -> Variable -> BoolOperator -> Variable -> BoolOperator -> Variable -> Description;
        mkDescription: Expression -> Description;
        Equals, IsNotEqual, And, Or, Minus: BoolOperator;
        TotalSupply, Supply, BalancesOwner, Balance, Allowed, Remaining, SenderAdress, SenderNewBalance, OldBalance, TransferredValue, RecipientAddress, TransferSuccessful, TransferNotSuccessful: Variable;
}

Defined ERC20

In [204]:
concrete ERC20Eng of ERC20 = {
    lincat
        Description = {str: Str};
        Variable = {name: Str; desc: Str};
        BoolOperator = {str: Str};
        Expression = {str: Str};
    lin
        Equals = {str = "should be equal to"};
        IsNotEqual = {str = "is not equal to"};
        And = {str = "and"};
        Or = {str = "or"};
        Minus = {str = "minus"};
        Supply = {name = "supply"; desc = "resulting total supply of tokens"};
        TotalSupply = {name = "_totalSupply"; desc = "total supply of tokens"};
        BalancesOwner = {name = "_balances[owner]"; desc = "resulting balance of the specified address"};
        Balance = {name = "balance"; desc = "balance of the specified address"};
        Allowed = {name = "_allowed[owner][spender]"; desc = "resulting allowance of the specified address"};
        Remaining = {name = "remaining"; desc = "allowance of the specified address"};
        SenderAdress = {name = "msg.sender"; desc = "sender's address"};
        SenderNewBalance = {name = "_balances[msg.sender]"; desc = "sender's new balance"};
        OldBalance = {name = "__verifier_old_uint"; desc = "old balance"};
        TransferredValue = {name = "value"; desc = "transferred value"};
        RecipientAddress = {name = "to"; desc = "recipient's address"};
        TransferSuccessful = {name = "success"; desc = "transfer is successful"};
        TransferNotSuccessful = {name = "success"; desc = "transfer is not successful"};
        mkExpression var1 op var2 = {str = "the" ++ var1.desc ++ op.str ++ "the" ++ var2.desc};
        mkConditionalExpression exp1 op1 exp2 op2 var1 op3 var2 op4 var3 = {str = "the" ++ exp1.str ++ op1.str ++ "their" ++ exp2.str ++ "if the" ++ var1.desc ++ op3.str ++ "the" ++ var2.desc ++ op2.str ++ "the" ++ var3.desc};
        mkDescription exp = exp;
}

Defined ERC20Eng

In [205]:
concrete ERC20Formal of ERC20 = {
    lincat
        Description = {str: Str};
        Variable = {name: Str; desc: Str};
        BoolOperator = {str: Str};
        Expression = {str: Str};
    lin
        Equals = {str = "=="};
        IsNotEqual = {str = "!="};
        And = {str = "&&"};
        Or = {str = "||"};
        Minus = {str = "-"};
        TotalSupply = {name = "_totalSupply"; desc = "_totalSupply"};
        Supply = {name = "supply"; desc = "supply"};
        BalancesOwner = {name = "_balances[owner]"; desc = "_balances[owner]"};
        Balance = {name = "balance"; desc = "balance"};
        Allowed = {name = "_allowed[owner][spender]"; desc = "_allowed[owner][spender]"};
        Remaining = {name = "remaining"; desc = "remaining"};
        SenderAdress = {name = "msg.sender"; desc = "msg.sender"};
        SenderNewBalance = {name = "_balances[msg.sender]"; desc = "_balances[msg.sender]"};
        OldBalance = {name = "__verifier_old_uint"; desc = "__verifier_old_uint"};
        TransferredValue = {name = "value"; desc = "value"};
        RecipientAddress = {name = "to"; desc = "to"};
        TransferSuccessful = {name = "success"; desc = "success"};
        TransferNotSuccessful = {name = "!success"; desc = "!success"};
        mkExpression var1 op var2 = {str = "@notice postcondition" ++ var1.desc ++ op.str ++ var2.desc};
        mkConditionalExpression exp1 op1 exp2 op2 var1 op3 var2 op4 var3 = {str = "@notice postcondition" ++ exp1.str ++ op1.str ++ exp2.str ++ "," ++ "then the" ++ var1.desc ++ op3.str ++ var2.desc ++ op2.str ++ var3.desc};
        mkDescription exp = exp;
}

Defined ERC20Formal

function totalSupply() external view returns (uint256);

In [206]:
parse -lang=ERC20Eng "the resulting total supply of tokens should be equal to the total supply of tokens" | linearize -lang=ERC20Formal

@notice postcondition supply == _totalSupply


function balanceOf(address who) external view returns (uint256);

In [207]:
parse -lang=ERC20Eng "the resulting balance of the specified address should be equal to the balance of the specified address" | linearize -lang=ERC20Formal

@notice postcondition _balances[owner] == balance


function allowance(address owner, address spender) external view returns (uint256);

In [208]:
parse -lang=ERC20Eng "the resulting allowance of the specified address should be equal to the allowance of the specified address" | linearize -lang=ERC20Formal

@notice postcondition _allowed[owner][spender] == remaining


In [209]:
parse -lang=ERC20Eng "the sender's new balance should be equal to their old balance if  the sender's address is equal to the recipient's address and the transfer is successful" | linearize -lang=ERC20Formal

In [210]:
parse -lang=ERC20Eng "the sender's new balance should be equal to their old balance if  the sender's address is equal to the recipient's address and the transfer is successful" | linearize

In [211]:
parse -lang=ERC20Eng "the resulting allowance of the specified address should be equal to the allowance of the specified address" | linearize

the resulting allowance of the specified address should be equal to the allowance of the specified address
@notice postcondition _allowed[owner][spender] == remaining
